In [0]:
import pandas as pd
import pymssql
import pymysql
import re

In [0]:
jdbcHostname = "54.245.193.229" #"54.245.193.229" 
jdbcDatabase = "DEV" #"KAGGLE" 
jdbcPort = "1433" #1433 
jdbcUsername = "sa"
jdbcPassword = "WindowsSQL-123"

host = "db4free.net"
db = "exocube"
uname = "sample_jenish"
pas = "Jenu7492@patel"

In [0]:
Source_Table_Name = "EMP_INFO"

In [0]:
def get_connection(jdbcHostname, jdbcUsername, jdbcPassword, jdbcDatabase):
    try:
        sql_connection = pymssql.connect(host=jdbcHostname,user=jdbcUsername,password=jdbcPassword,database=jdbcDatabase, as_dict=True, autocommit=True)
#This exception is raised for things that are not in control of the programmer. For e.g unexpected disconnect, error in memory allocation etc, selected database not exists.
    except pymssql.OperationalError as e:
        print("connection failed")
    except:
        print("Unknown error occurred")
    else:
        return sql_connection

In [0]:
def get_myconnection(host, uname, pas, db):
    try:
        sql_myconnection = pymysql.connect(host=host,user=uname,passwd=pas,db=db, cursorclass=pymysql.cursors.DictCursor, autocommit=True)
#This exception is raised for things that are not in control of the programmer. For e.g unexpected disconnect, error in memory allocation etc, selected database not exists.
    except pymysql.OperationalError as e:
        if e.args[0] == 2003:
            print('server not found')
        elif e.args[0] == 1044:
            print('database not found')
        elif e.args[0] == 1045:
            print('username or password incorrect')
    except:
        print("Unknown error occurred")   
    else:
        return sql_myconnection

In [0]:
def display_mssql_data(sql_disqry, sql_conn):
    try:
        cursor = sql_conn.cursor()
        cursor.execute(sql_disqry)
        count = cursor.rowcount
        if count:
            sql_data = pd.DataFrame(cursor.fetchall())
            spark_df = spark.createDataFrame(sql_data)
            dfres = spark_df.toPandas()
            return dfres
        else:
            print('No Record')
    except pymssql.ProgrammingError as e:
        if e.args[0] == 208:
            print('table doesnt exist')
        elif e.args[0] == 102:
            print('sql query syntax error')  
    finally:
        sql_conn.close()

In [0]:
def sql_auto_gen_qry(qry, sql_conn):
    try:
        adres = 'select count(*) as CNT,'
        cursor = sql_conn.cursor()
        cursor.execute(qry)
        count = cursor.rowcount
        if count:
            sql_data = cursor.fetchall()
            for val in sql_data:
                dt,col = val.values()
                dtyp = " ".join(re.split("[^a-zA-Z]", dt))
                if dtyp == 'int':
                    qry = "min("+col+")as "+col+"MIN,max("+col+")as "+col+"MAX,sum("+col+")as "+col+"SUM,avg("+col+")as "+col+"AVG,"
                elif dtyp == 'varchar':
                    qry = "min(trim(lower("+col+")))as "+col+"MIN,max(trim(lower("+col+")))as "+col+"MAX,sum(len(trim(lower("+col+"))))as "+col+"SUM,avg(len(trim(lower("+col+"))))as "+col+"AVG,"
                elif dtyp == 'date':
                    qry = "min(DATEDIFF(s, '1970-01-01 00:00:00', "+col+"))as "+col+"MIN,max(DATEDIFF(s, '1970-01-01 00:00:00', "+col+"))as "+col+"MAX,sum(DATEDIFF(s, '1970-01-01 00:00:00', "+col+") * convert(bigint, 1000))as "+col+"SUM,avg(DATEDIFF(s, '1970-01-01 00:00:00', "+col+") * convert(bigint, 1000))as "+col+"AVG,"
                else:
                    print("no datatype found")
                adres += qry
            rmres = adres.rstrip(',')
            sql_disqry = ''.join((rmres," from "+Source_Table_Name+""))
            return sql_disqry
        else:
            print('No Record')
    except pymssql.ProgrammingError as e:
        if e.args[0] == 208:
            print('table doesnt exist')
        elif e.args[0] == 102:
            print('sql query syntax error')  
    finally:
        sql_conn.close()

In [0]:
sql_conn = get_connection(jdbcHostname, jdbcUsername, jdbcPassword, jdbcDatabase)
qry ="(SELECT DATA_TYPE, COLUMN_NAME FROM information_schema.columns where table_name = '{}')".format(Source_Table_Name)
sql_disqry = sql_auto_gen_qry(qry, sql_conn)

In [0]:
sql_conn = get_connection(jdbcHostname, jdbcUsername, jdbcPassword, jdbcDatabase)
dfresult = display_mssql_data(sql_disqry, sql_conn)
#displayHTML(htmlcode)
import itertools
col = list(dfresult)
list_of_val = dfresult.values
val = list_of_val.tolist()
df = pd.DataFrame(list(zip(col, val[0])),columns =['Name', 'val'])
conhtml = df.to_html()
rephtml = finres.replace('<table border="1" class="dataframe">', '<head> <style> th, td {border-style: outset;}</style></head><table style="width:100%">')
repfinhtml = rephtml.replace('<tr style="text-align: right;">', '<tr style="text-align: center;">')
displayHTML(repfinhtml)

,Name,val
0,CNT,15
1,Emp_IdMIN,4
2,Emp_IdMAX,21
3,Emp_IdSUM,194
4,Emp_IdAVG,12
5,Emp_nameMIN,amit desai
6,Emp_nameMAX,suresh ahir
7,Emp_nameSUM,175
8,Emp_nameAVG,11
9,DesignationMIN,admin


In [0]:
def display_mysql_data(sql_disqry, sql_conn):
    try:
        cursor = sql_conn.cursor()
        sql_exec = cursor.execute(sql_disqry)
        if sql_exec:
            sql_data = pd.DataFrame(cursor.fetchall())
            spark_df = spark.createDataFrame(sql_data)
            dfmyres = spark_df.toPandas()
            return dfmyres
        else:
            print(sql_exec)
            print("No Record")
    except pymysql.ProgrammingError as e:
        if e.args[0] == 1146:
            print('table doesnt exist')
        elif e.args[0] == 1064:
            print('sql query syntax error')  
    finally:
        sql_conn.close()

In [0]:
def mysql_auto_gen_qry(qry, sql_conn):
    try:
        adres = 'select count(*) as CNT,'
        cursor = sql_conn.cursor()
        cursor.execute(qry)
        count = cursor.rowcount
        if count:
            sql_data = cursor.fetchall()
            for val in sql_data:
                dt,col = val.values()
                dtyp = " ".join(re.split("[^a-zA-Z]", dt))
                if dtyp == 'int':
                    qry = "min("+col+")as "+col+"MIN,max("+col+")as "+col+"MAX,sum("+col+")as "+col+"SUM,FLOOR(avg("+col+"))as "+col+"AVG,"
                elif dtyp == 'varchar':
                    qry = "min(trim(lower("+col+")))as "+col+"MIN,max(trim(lower("+col+")))as "+col+"MAX,sum(length(trim(lower("+col+"))))as "+col+"SUM,FLOOR(avg(length(trim(lower("+col+")))))as "+col+"AVG,"
                elif dtyp == 'date':
                    qry = "min(UNIX_TIMESTAMP("+col+"))as "+col+"MIN,max(UNIX_TIMESTAMP("+col+"))as "+col+"MAX,sum(UNIX_TIMESTAMP("+col+"))as "+col+"SUM,FLOOR(avg(UNIX_TIMESTAMP("+col+")))as "+col+"AVG,"
                else:
                    print("no datatype found")
                adres += qry
            rmres = adres.rstrip(',')
            sql_disqry = ''.join((rmres," from "+Source_Table_Name+""))
            return sql_disqry
        else:
            print('No Record')
    except pymssql.ProgrammingError as e:
        if e.args[0] == 208:
            print('table doesnt exist')
        elif e.args[0] == 102:
            print('sql query syntax error')  
    finally:
        sql_conn.close()

In [0]:
sql_conn = get_myconnection(host, uname, pas, db)
qry ="(SELECT DATA_TYPE, COLUMN_NAME FROM information_schema.columns where table_name = '{}')".format(Source_Table_Name)
sql_disqry = mysql_auto_gen_qry(qry, sql_conn)

In [0]:
sql_conn = get_myconnection(host, uname, pas, db)
dfmyresult = display_mysql_data(sql_disqry, sql_conn)
mycol = list(dfmyresult)
list_of_val = dfmyresult.values
myval = list_of_val.tolist()
df = pd.DataFrame(list(zip(mycol, myval[0])),columns =['Name', 'val'])
conhtml = df.to_html()
rephtml = finres.replace('<table border="1" class="dataframe">', '<head> <style> th, td {border-style: outset;}</style></head><table style="width:100%">')
repfinhtml = rephtml.replace('<tr style="text-align: right;">', '<tr style="text-align: center;">')
displayHTML(repfinhtml)

/databricks/spark/python/pyspark/sql/pandas/utils.py:79: UserWarning: The conversion of DecimalType columns is inefficient and may take a long time. Column names: [CitySUM, DepartmentSUM, DesignationSUM, Emp_IdSUM, Emp_nameSUM, Joining_DateSUM, Joining_DateAVG, SalarySUM] If those columns are not necessary, you may consider dropping them or converting to primitive types before the conversion.
  warnings.warn(


,Name,val
0,CNT,15
1,Emp_IdMIN,4
2,Emp_IdMAX,21
3,Emp_IdSUM,194
4,Emp_IdAVG,12
5,Emp_nameMIN,amit desai
6,Emp_nameMAX,suresh ahir
7,Emp_nameSUM,175
8,Emp_nameAVG,11
9,DesignationMIN,admin


In [0]:
def display_target_data(sql_disqry):
    try:
            pdf = spark.sql(sql_disqry)
            dftgres = pdf.toPandas()
            return dftgres
    except pymssql.ProgrammingError as e:
        if e.args[0] == 208:
            print('table doesnt exist')
        elif e.args[0] == 102:
            print('sql query syntax error')  

In [0]:
def tar_auto_gen_qry(qry):
    try:
        adres = 'select count(*) as CNT,'
        pd_df = spark.sql(qry)
        ow_list = pd_df.select('col_name', 'data_type').collect()
        sql_data = ow_list[:-3]
        if sql_data:
            for val in sql_data:
                col,dt = val
                dtyp = " ".join(re.split("[^a-zA-Z]", dt))
                if dtyp == 'bigint':
                    qry = "min("+col+")as "+col+"MIN,max("+col+")as "+col+"MAX,sum("+col+")as "+col+"SUM,FLOOR(avg("+col+"))as "+col+"AVG,"
                elif dtyp == 'string':
                    qry = "min(trim(lower("+col+")))as "+col+"MIN,max(trim(lower("+col+")))as "+col+"MAX,sum(length(trim(lower("+col+"))))as "+col+"SUM,FLOOR(avg(length(trim(lower("+col+")))))as "+col+"AVG,"
                elif dtyp == 'date':
                    qry = "min(UNIX_TIMESTAMP("+col+"))as "+col+"MIN,max(UNIX_TIMESTAMP("+col+"))as "+col+"MAX,sum(UNIX_TIMESTAMP("+col+"))as "+col+"SUM,FLOOR(avg(UNIX_TIMESTAMP("+col+")))as "+col+"AVG,"
                else:
                    print("no datatype found")
                adres += qry
            rmres = adres.rstrip(',')
            sql_disqry = ''.join((rmres," from "+Source_Table_Name+""))
            return sql_disqry
        else:
            print('No Record')
    except pymssql.ProgrammingError as e:
        if e.args[0] == 208:
            print('table doesnt exist')
        elif e.args[0] == 102:
            print('sql query syntax error')  

In [0]:
qry =("DESCRIBE TABLE {}").format(Source_Table_Name)
sql_disqry = tar_auto_gen_qry(qry)

In [0]:
dftgresult = display_target_data(sql_disqry)
tgcol = list(dftgresult)
list_of_val = dftgresult.values
tgval = list_of_val.tolist()
df = pd.DataFrame(list(zip(tgcol, tgval[0])),columns =['Name', 'val'])
conhtml = df.to_html()
rephtml = finres.replace('<table border="1" class="dataframe">', '<head> <style> th, td {border-style: outset;}</style></head><table style="width:100%">')
repfinhtml = rephtml.replace('<tr style="text-align: right;">', '<tr style="text-align: center;">')
displayHTML(repfinhtml)

,Name,val
0,CNT,15
1,Emp_IdMIN,4
2,Emp_IdMAX,21
3,Emp_IdSUM,194
4,Emp_IdAVG,12
5,Emp_nameMIN,amit desai
6,Emp_nameMAX,suresh ahir
7,Emp_nameSUM,175
8,Emp_nameAVG,11
9,DesignationMIN,admin
